Check your current context to make sure you are targeting the right Kubernetes cluster

In [1]:
!kubectl config current-context

twright-arc-demo-3


Check the labels currently on your nodes

In [2]:
!kubectl get nodes --show-labels

NAME                                STATUS   ROLES   AGE    VERSION    LABELS
aks-nodepool1-25819688-vmss000000   Ready    agent   4d4h   v1.15.10   agentpool=nodepool1,beta.kubernetes.io/arch=amd64,beta.kubernetes.io/instance-type=Standard_D8_v3,beta.kubernetes.io/os=linux,failure-domain.beta.kubernetes.io/region=eastus,failure-domain.beta.kubernetes.io/zone=0,kubernetes.azure.com/cluster=MC_twright-arc-demo-3_twright-arc-demo-3_eastus,kubernetes.azure.com/role=agent,kubernetes.io/arch=amd64,kubernetes.io/hostname=aks-nodepool1-25819688-vmss000000,kubernetes.io/os=linux,kubernetes.io/role=agent,node-role.kubernetes.io/agent=,storageprofile=managed,storagetier=Standard_LRS
aks-nodepool1-25819688-vmss000001   Ready    agent   4d4h   v1.15.10   agentpool=nodepool1,beta.kubernetes.io/arch=amd64,beta.kubernetes.io/instance-type=Standard_D8_v3,beta.kubernetes.io/os=linux,failure-domain.beta.kubernetes.io/region=eastus,failure-domain.beta.kubernetes.io/zone=1,kubernetes.azure.com/cluster=MC_

Label a node

In [3]:
#kubectl label node <your-node-name> <name>=<value>
!kubectl label node aks-nodepool1-25819688-vmss000000 test=twright

node/aks-nodepool1-25819688-vmss000000 labeled


Provision a pod that is targeted at the node labeled with the label.  Save the yaml below as a .yaml file.  If you are using this from the twright-msft/k8s-toolkit this file is already created at ./yaml/hello-world-node-targeting.yaml.

```yaml
apiVersion: v1
kind: Pod
metadata:
  name: hello-world-node-targeting
spec:
  containers:
  - name: hello-world-node-targeting
    image: alpine
    command: ['sh', '-c', 'echo Hello World! && sleep 600']
  nodeSelector:
    test: twright
```

Apply the yaml file to your Kubernetes cluster.

In [5]:
!kubectl apply -f ./yaml/hello-world-node-targeting.yaml

pod/hello-world-node-targeting created


Check to see if the pod was provisioned

In [6]:
!kubectl get po

NAME                         READY   STATUS    RESTARTS   AGE
hello-world                  1/1     Running   0          4m52s
hello-world-node-targeting   1/1     Running   0          5s


Check which node the pod landed on.  It should be the node you labeled above.

In [7]:
!kubectl describe po/hello-world-node-targeting

Name:               hello-world-node-targeting
Namespace:          default
Priority:           0
PriorityClassName:  <none>
Node:               aks-nodepool1-25819688-vmss000000/10.240.0.4
Start Time:         Sat, 04 Apr 2020 14:49:48 -0700
Labels:             <none>
Annotations:        kubectl.kubernetes.io/last-applied-configuration:
                      {"apiVersion":"v1","kind":"Pod","metadata":{"annotations":{},"name":"hello-world-node-targeting","namespace":"default"},"spec":{"container...
Status:             Running
IP:                 10.244.1.31
Containers:
  hello-world-node-targeting:
    Container ID:  docker://b76bcae48defb29a417179ae64d972e3ae229706e6299efc6d56e3cdfb1875a9
    Image:         alpine
    Image ID:      docker-pullable://alpine@sha256:b276d875eeed9c7d3f1cfa7edb06b22ed22b14219a7d67c52c56612330348239
    Port:          <none>
    Host Port:     <none>
    Command:
      sh
      -c
      echo Hello World! && sleep 600
    State:          Running
      Started

Remove the label

In [8]:
#kubectl label node <your-node-name> <name>-
!kubectl label node aks-nodepool1-25819688-vmss000000 test-

node/aks-nodepool1-25819688-vmss000000 labeled


Check to see if the label is removed

In [9]:
!kubectl get nodes --show-labels

NAME                                STATUS   ROLES   AGE    VERSION    LABELS
aks-nodepool1-25819688-vmss000000   Ready    agent   4d4h   v1.15.10   agentpool=nodepool1,beta.kubernetes.io/arch=amd64,beta.kubernetes.io/instance-type=Standard_D8_v3,beta.kubernetes.io/os=linux,failure-domain.beta.kubernetes.io/region=eastus,failure-domain.beta.kubernetes.io/zone=0,kubernetes.azure.com/cluster=MC_twright-arc-demo-3_twright-arc-demo-3_eastus,kubernetes.azure.com/role=agent,kubernetes.io/arch=amd64,kubernetes.io/hostname=aks-nodepool1-25819688-vmss000000,kubernetes.io/os=linux,kubernetes.io/role=agent,node-role.kubernetes.io/agent=,storageprofile=managed,storagetier=Standard_LRS
aks-nodepool1-25819688-vmss000001   Ready    agent   4d4h   v1.15.10   agentpool=nodepool1,beta.kubernetes.io/arch=amd64,beta.kubernetes.io/instance-type=Standard_D8_v3,beta.kubernetes.io/os=linux,failure-domain.beta.kubernetes.io/region=eastus,failure-domain.beta.kubernetes.io/zone=1,kubernetes.azure.com/cluster=MC_

Check to see if the pod has been removed.  It probbably won't be.

In [10]:
!kubectl get po

NAME                         READY   STATUS    RESTARTS   AGE
hello-world                  1/1     Running   0          5m18s
hello-world-node-targeting   1/1     Running   0          31s


Delete the pod

In [11]:
!kubectl delete -f ./yaml/hello-world-node-targeting.yaml

pod "hello-world-node-targeting" deleted


Check the pod again

In [12]:
!kubectl get po

NAME          READY   STATUS    RESTARTS   AGE
hello-world   1/1     Running   0          6m39s


Try to create it again

In [14]:
!kubectl create -f ./yaml/hello-world-node-targeting.yaml

pod/hello-world-node-targeting created


Check to see if the pod gets deployed.  Shouldn't.

In [15]:
!kubectl get po

NAME                         READY   STATUS    RESTARTS   AGE
hello-world                  1/1     Running   0          6m58s
hello-world-node-targeting   0/1     Pending   0          2s


Check the reason it is in pending.  Should be because there is no node that matches the nodeSelector.

In [17]:
!kubectl describe po/hello-world-node-targeting

Name:               hello-world-node-targeting
Namespace:          default
Priority:           0
PriorityClassName:  <none>
Node:               <none>
Labels:             <none>
Annotations:        <none>
Status:             Pending
IP:                 
Containers:
  hello-world-node-targeting:
    Image:      alpine
    Port:       <none>
    Host Port:  <none>
    Command:
      sh
      -c
      echo Hello World! && sleep 600
    Environment:  <none>
    Mounts:
      /var/run/secrets/kubernetes.io/serviceaccount from default-token-6skh8 (ro)
Conditions:
  Type           Status
  PodScheduled   False 
Volumes:
  default-token-6skh8:
    Type:        Secret (a volume populated by a Secret)
    SecretName:  default-token-6skh8
    Optional:    false
QoS Class:       BestEffort
Node-Selectors:  test=twright
Tolerations:     node.kubernetes.io/not-ready:NoExecute for 300s
                 node.kubernetes.io/unreachable:NoExecute for 300s
Events:
  Type     Reason            Age         